# Challenge: Deploy a simple RAG Application using an API
In this challenge you will have to build an RAG application that can answer questions using data on a webpage. You will be using Graphlit to build this application since they provide a hosted solution to ingest data and build chatbots on top of it.

[![open in colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/LinkedInLearning/generative-ai-and-llmops-deploying-and-managing-llms-in-production-4465782/blob/main/ch-03/challenge_deploy_RAG_using_API.ipynb)

In [ ]:
!pip install graphlit-client -q

In [ ]:
from typing import Optional
from graphlit import Graphlit
from graphlit_api import *

In [ ]:
# Get your Graphlit token from https://portal.graphlit.dev
graphlit = Graphlit(organization_id="insertkey", environment_id="insertkey", jwt_secret="insertkey")

In [ ]:
# Function to create a new feed using the provided URL (uri) and return the feed ID or an error message.
async def create_feed(graphlit, uri):
    input = FeedInput(
        name=uri,  # The name of the feed is set to the URL itself.
        type=FeedTypes.WEB,  # Specifies that the feed type is a web feed.
        web=WebFeedPropertiesInput(
            uri=uri,  # URL for the web feed.
            readLimit=10  # Sets the limit on the number of reads allowed.
        )
    )

    try:
        # Attempt to create the feed using the graphlit client.
        response = await graphlit.client.create_feed(input)

        # Extract and return the feed ID from the response.
        feed_id = response.create_feed.id
    except GraphQLClientError as e:
        # Return the error message in case of a GraphQL client error.
        return str(e)

    return feed_id

# Function to create a specification for a summarization task using the ANTHROPIC model and return the specification ID or an error message.
async def create_specification(graphlit):
    input = SpecificationInput(
        name="Summarization",  # Name of the specification.
        type=SpecificationTypes.COMPLETION,  # Specifies that this is a completion type specification.
        serviceType=ModelServiceTypes.ANTHROPIC,  # Sets the model service type to Anthropic.
        searchType=SearchTypes.VECTOR,  # Sets the search type to vector.
        anthropic=AnthropicModelPropertiesInput(
            model=AnthropicModels.CLAUDE_3_HAIKU,  # Specifies the Anthropic model to be used.
            temperature=0.1,  # Sets the temperature for model output, affecting randomness.
            probability=0.2,  # Sets the probability threshold for certain responses.
            completionTokenLimit=2048,  # Limits the number of tokens in the completion output.
        )
    )

    try:
        # Attempt to create the specification using the graphlit client.
        response = await graphlit.client.create_specification(input)

        # Extract and return the specification ID from the response.
        spec_id = response.create_specification.id
    except GraphQLClientError as e:
        # Return the error message in case of a GraphQL client error.
        return str(e)

    return spec_id

# Function to create a new conversation using a specification ID and return the conversation ID or an error message.
async def create_conversation(graphlit, spec_id):
    input = ConversationInput(
        name="Conversation",  # Name of the conversation.
        specification=EntityReferenceInput(
            id=spec_id  # Reference to the specification ID to be used for this conversation.
        )
    )

    try:
        # Attempt to create the conversation using the graphlit client.
        response = await graphlit.client.create_conversation(input)

        # Extract and return the conversation ID from the response.
        conv_id = response.create_conversation.id
    except GraphQLClientError as e:
        # Return the error message in case of a GraphQL client error.
        return str(e)

    return conv_id

# Function to send a prompt to an existing conversation and return the generated message and any citations, or an error message.
async def prompt_conversation(graphlit, conv_id, prompt):
    try:
        # Attempt to send a prompt to the conversation using the graphlit client.
        response = await graphlit.client.prompt_conversation(prompt, conv_id)

        # Extract the generated message and citations from the response.
        message = response.prompt_conversation.message.message
        citations = response.prompt_conversation.message.citations

        return message, citations
    except GraphQLClientError as e:
        # Return None for message and the error message in case of a GraphQL client error.
        return None, str(e)

In [ ]:
# Set the URL for the feed input, using a Wikipedia page as the source.
uri = 'https://en.wikipedia.org/wiki/The_Lord_of_the_Rings'

# Call the create_feed function with the graphlit client and the URL, asynchronously creating a new web feed.
feed_id = await create_feed(graphlit, uri)

# Print the created feed ID to verify the feed has been created successfully.
feed_id

'1768fd8f-5512-4aaf-a746-4b491723f26b'

In [ ]:
# Call the create_specification function with the graphlit client, asynchronously creating a new specification.
spec_id = await create_specification(graphlit)

# Print the created specification ID to verify the specification has been created successfully.
spec_id

'91013da5-e41f-4f13-8e19-99ff666feb42'

In [ ]:
# Create a new conversation using the created specification ID.
# The specification ID is passed as an argument to link the conversation to the specific model configuration.
conv_id = await create_conversation(graphlit, spec_id)

# Print the created conversation ID to verify the conversation has been created successfully.
conv_id

'2bcabcfe-d99d-400f-922d-4557ce186f5c'

In [ ]:
# Send a prompt to the created conversation, asking it to "Summarize this webpage?".
# The prompt and conversation ID are passed as arguments to generate a response based on the specified model and feed.
response = await prompt_conversation(graphlit, conv_id, "Summarize this webpage?")

# Print the response containing the generated message and any citations from the summarization.
response

('The Lord of the Rings is a classic fantasy novel by J.R.R. Tolkien, first published in the 1950s. It has become one of the most popular and influential works of fiction in the 20th century, with its epic story, detailed world-building, and memorable characters.\n\nThe novel was initially published in three volumes - The Fellowship of the Ring, The Two Towers, and The Return of the King. It follows the journey of the hobbit Frodo Baggins as he and a fellowship attempt to destroy the powerful One Ring and defeat the dark lord Sauron.\n\nThe Lord of the Rings has had a lasting impact on the fantasy genre, inspiring numerous adaptations in film, television, video games, and other media. It is considered a landmark work that has shaped modern fantasy storytelling.\n\nThe novel has received both critical acclaim and some criticism over the years, with some praising its depth and themes, while others have noted issues like a lack of psychological depth or a conservative worldview. However, 

In [ ]:
response = await prompt_conversation(graphlit, conv_id, "Who are te main characters?")
response

("The main characters in The Lord of the Rings are Frodo Baggins, the hobbit who is tasked with destroying the powerful One Ring, and his companions in the Fellowship - Aragorn, the heir to the throne of Gondor; Gandalf, the wizard who guides the group; Legolas the elf; Gimli the dwarf; and the hobbits Samwise Gamgee, Meriadoc 'Merry' Brandybuck, and Peregrin 'Pippin' Took.\n\nOther important characters include Sauron, the dark lord who seeks to reclaim the One Ring, and his servants like the Nazgûl; Galadriel and Elrond, powerful elven leaders; Théoden, the king of Rohan; and Faramir, the steward of Gondor.",
 None)